In [12]:
import pandas as pd
import os

from langchain_qdrant import QdrantVectorStore

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [13]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "nlpai-lab/KURE-v1"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

/Users/hongbikim/anaconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
client = QdrantClient(
    host="localhost",
    port=6333,
    # path = "/Users/hongbikim/Dev/dacon_finance"
)

collections = client.get_collections()
collections

CollectionsResponse(collections=[CollectionDescription(name='rag-finance')])

In [15]:
vector_store = QdrantVectorStore(
    client = client,
    collection_name = "rag-finance",
    embedding = hf_embeddings
)

In [2]:
test = pd.read_csv("/Users/hongbikim/Dev/dacon_finance/open/test.csv")
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [4]:
for qq in test['Question']:
    if '\n' not in qq:
        print(qq)

트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표를 설명하세요.
전자금융거래법에 따라 이용자가 금융 분쟁조정을 신청할 수 있는 기관을 기술하세요.
딥페이크 기술 악용에 대비한 금융권의 대응 방안을 기술하세요.
디지털 지갑(Digital Wallet)에서 우려되는 주요 보안 위협을 설명하세요.
개인정보 접근 권한 검토는 어떻게 수행해야 하며, 그 목적은 무엇인가요?
정보보호의 3대 요소에 해당하는 보안 목표를 3가지 기술하세요.
금융회사가 정보보호 예산을 관리할 때, 전자금융감독규정상 정보기술부문 인력 및 예산의 기준 비율은 얼마인가요?
SMTP 프로토콜의 보안상 주요 역할을 설명하세요.
ID/PW 기반 인증 환경에서 발생할 수 있는 대표적인 공격 유형을 2가지 이상 서술하고 대응 방안을 설명하세요.
전자자금이체의 지급 효력 발생 시점을 전자금융거래법 기준에 따라 설명하세요.
DNSSEC의 주요 기능과 이를 통해 해결할 수 있는 보안 위협을 설명하세요.
VLAN(Virtual LAN)의 주요 기능을 설명하세요.
개인신용평점 하락 가능성과 관련해 설명의무가 발생하는 경우, 금융기관이 설명해야 할 주요 내용은 무엇인가요?
인증 및 권한관리 측면에서 특수 계정에 대한 보안조치 방안은 무엇인가요?
전자금융데이터 보안에서 암호화 방식을 선택할 때 고려해야 할 주요 기준은 무엇인가요?


In [16]:
query = test['Question'][0].split("\n")[0]

results = vector_store.similarity_search(
    query = query,
    k = 10,
    # filter = models.Filter(must=[models.FieldCondition(
    #     key = "metadata.page", # 특정 필드(예: page) 기반 필터링
    #     match = models.MatchValue(value=2), # page 1인 문서만 검색
    # )])
)

In [17]:
results

[Document(metadata={'file_name': '자본시장과 금융투자업에 관한 법률 시행령(대통령령)(제35587호)(20250903).pdf', 'file_id': 'df6aeb0c59fd4235bebe1ae3a96799bd', 'page_chunk_id': 'page_19_chunk_0', 'chunk_length': 339, '_id': '2f77f108-5802-4fb3-8f7c-15faf52c8b4c', '_collection_name': 'rag-finance'}, page_content='법제처 20 국가법령정보센터\n자본시장과 금융투자업에 관한 법률 시행령\n1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목의 금융투자업의 경우: 해당 금융투자업에 속하는 금융투자상품\n중 증권, 장내파생상품, 장외파생상품 각각을 기준으로 하여 인가받지 않은 다른 업무 단위\n가. 투자매매업\n나. 투자매매업(인수업은 제외한다)\n다. 투자매매업(인수업만 해당한다)\n2. 별표 1에 따른 금융투자업의 종류 중 투자중개업(같은 표에 따라 인가업무 단위가 2l-1-1, 2l-1-2, 2-14-1 및 2-\n14-2인 투자중개업은 제외한다)의 경우: 투자중개업에 속하는 인가업무 단위 중 인가받지 않은 다른 업무 단위'),
 Document(metadata={'file_name': '자본시장과 금융투자업에 관한 법률 시행령(대통령령)(제35471호)(20250722).pdf', 'file_id': 'fc003e50e40240ff9ba53360eab09a68', 'page_chunk_id': 'page_19_chunk_0', 'chunk_length': 339, '_id': '581051e1-a545-4d63-8728-74eb02e5316a', '_collection_name': 'rag-finance'}, page_content='법제처 20 국가법령정보센터\n자본시장과 금융투자업에 관한 법률 시행령\n1. 별표 1에 따른 금융투자업의 종류 중 다음 각 목